In [8]:
import random

class OdsekomaLinearnaFunkcija:
    """
    Razred za odsekoma linearne funkcije.
    """

    def __init__(self, tocke):
        """
        Konstruktor objekta.
        """
        tocke = dict(tocke) # naredimo nov slovar s točkami
        self.prelomi = sorted(tocke.keys()) # urejen seznam točk preloma
        self.vrednosti = [tocke[x] for x in self.prelomi] # seznam ustreznih vrednosti

    def __call__(self, x):
        """
        Evaluacija funkcije v točki x.
        """
        if x <= self.prelomi[0]: # levo od prve točke vrnemo prvo vrednost
            return self.vrednosti[0]
        elif x >= self.prelomi[-1]: # desno od zadnje točke vrnemo zadnjo vrednost
            return self.vrednosti[-1]
        i, j = 0, len(self.prelomi) - 1 # delamo bisekcijo
        while i < j - 1:
            h = (i + j) // 2
            if x < self.prelomi[h]:
                j = h
            else:
                i = h
        return ((self.prelomi[j] - x) * self.vrednosti[i] + (x - self.prelomi[i]) * self.vrednosti[j])  / (self.prelomi[j] - self.prelomi[i])

class TrikotnaFunkcija(OdsekomaLinearnaFunkcija):
    def __init__(self, a, b, c, h):
        """
        Konstruktor za trikotno funkcijo z vznožjem pri a in c ter vrhom pri b z vrednostjo h
        """
        assert a < b < c
        super().__init__({a: 0, b: h, c: 0})

    @staticmethod
    def generiraj(range_x=(0, 1), range_y=(0, 100)):
        """
        Generator funkcij pri podanih rangih vrednosti.
        """
        a, b, c = sorted(random.uniform(*range_x) for _ in range(3))
        h = random.uniform(*range_y)
        return TrikotnaFunkcija(a, b, c, h)

class SimetricnaTrikotnaFunkcija(OdsekomaLinearnaFunkcija):
    def __init__(self, a, d, c, h):
        """
        Konstruktor za trikotno funkcijo z vznožjem pri a in c ter vrhom pri d = (c-a)/2 z vrednostjo h
        """
        assert a < d < c
        super().__init__({a: 0, d: h, c: 0})

    @staticmethod
    def generiraj(range_x=(0, 1), range_y=(0, 100)):
        """
        Generator funkcij pri podanih rangih vrednosti.
        """
        a, d, c = sorted(random.uniform(*range_x) for _ in range(3))
        h = random.uniform(*range_y)
        return SimetricnaTrikotnaFunkcija(a, d, c, h)

#######################################################################################################################################################

# Nastavitve programa 
# Tukaj nastavimo število trikotnih funkcij(n) in število trikotnih funkcij v podmnožici(z), z < n
# Nastavimo zaprt interval [a,c], a < c

n = 10
z = 7
a = 0
c = 1
d = (c-a)/2

if n <= 0:
    print("Napaka: število trikotnih funkcij mora biti pozitivno")
elif n < z:
    print("Napaka: števlo vseh trikotnih funkcij mora biti večje kot število v podmnožici")
elif a >= c :
    print("Napaka: upoštevaj pogoj a < b")
else:
    print("Uspešno nastavljene nastavitve")


seznam_funkcij = [TrikotnaFunkcija.generiraj(range_x=(-1, 1), range_y=(0, 10)) for i in range(n)]
#seznam_simetricnih_funkcij = [SimetricnaTrikotnaFunkcija.generiraj(range_x=(-1, 1), range_y=(0, 10)) for _ in range(n)]
#seznam_trikotne_tocke = sorted(set(x for f in seznam_simetricnih_funkcij for x in f.prelomi))
seznam_tock = sorted(set(x for f in seznam_funkcij for x in f.prelomi))

#######################################################################################################################################################
def vsota_trikotnih_funkcij(seznam_funkcij, seznam_tock):
    k = len(seznam_tock)
    n = len(seznam_funkcij)

    p = MixedIntegerLinearProgram(maximization=False)
    v = p.new_variable(binary=True)
    M = p.new_variable(integer=False, nonnegative=True)
    m = p.new_variable(integer=False, nonnegative=True)
    for i in seznam_tock:
        p.add_constraint(sum(seznam_funkcij[j](x[i]) * v[j] for j in range(1, n) <= M))
        p.add_constraint(sum(seznam_funkcij[j](x[i]) * v[j] for j in range(1, n) >= m))
    p.add_constraint(sum(v[j] for j in range(1,n) >= 1))
    p.add_constraint(sum(v[j] for j in range(1,n) <= z))
    p.set_objective(M-m)

    vsota = p.solve()
    M, m = p.get_values(M), p.get_values(m)
    vkljucene = []
    for j in range(1,n):
        if v[j] == 1:
            vkljucene.append(seznam_funkcij[j])
    return [vsota, vkljucene]


vsota_trikotnih_funkcij(seznam_funkcij, seznam_tock)



Uspešno nastavljene nastavitve


TypeError: '<=' not supported between instances of 'range' and 'sage.numerical.mip.MIPVariable'